# Calculate median weekly trends in snowlines for all sites

In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys
from scipy.stats import iqr
from shapely import wkt
import seaborn as sns
import contextily as ctx
import numpy as np

## Define paths in directory, import functions

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
figures_out_path = os.path.join(base_path, 'figures')
study_sites_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/'

## Load compiled glacier boundaries (AOIs) and snowlines

In [ ]:
# -----Load AOIs
aois_path = os.path.join(scm_path, 'all_AOIs')
aois_fn = 'all_aois.shp'
aois = gpd.read_file(os.path.join(aois_path, aois_fn))
print('All glacier boundaries loaded from file.')

# -----Load snowlines
snowlines_path = os.path.join(scm_path, 'all_snowlines')
snowlines_fn = 'all_snowlines.csv'
snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))
print('All snowlines loaded from file.')

## Calculate weekly median trends for each site

In [ ]:
snowlines_medians_fn = os.path.join(scm_path, 'results', 'all_snowlines_weekly_median_trends.csv') 
if not os.path.exists(snowlines_medians_fn):
    # add week-of-year (WOY column to snowlines
    snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
    snowlines['WOY'] = snowlines['datetime'].dt.isocalendar().week
    # determine columns to calculate weekly stats
    columns = ['AAR', 'snowline_elevs_median_m', 'SCA_m2', 'ELA_from_AAR_m']
    snowlines_medians = pd.DataFrame()
    for site_name in tqdm(snowlines['site_name'].drop_duplicates().values):
        # subset snowlines to site
        snowlines_site = snowlines.loc[snowlines['site_name']==site_name]
        # calculate weekly quartile trends
        q1 = snowlines_site[['WOY'] + columns].groupby(by='WOY').quantile(0.25)
        q1.columns = [x + '_P25' for x in q1.columns]
        q2 = snowlines_site[['WOY'] + columns].groupby(by='WOY').quantile(0.5)
        q2.columns = [x + '_P50' for x in q2.columns]
        q3 = snowlines_site[['WOY'] + columns].groupby(by='WOY').quantile(0.75)
        q3.columns = [x + '_P75' for x in q3.columns]
        qs = pd.merge(q1, pd.merge(q2, q3, on='WOY'), on='WOY')
        qs = qs.reindex(sorted(qs.columns), axis=1)
        qs['WOY'] = qs.index
        qs['site_name'] = site_name
        # concatenate to medians dataframe
        snowlines_medians = pd.concat([snowlines_medians, qs])
    # save to file
    snowlines_medians.to_csv(snowlines_medians_fn, index=False)
    print('Median weekly snow trends saved to file: ', snowlines_medians_fn)
        
else:
    snowlines_medians = pd.read_csv(snowlines_medians_fn)
    print('Median weekly snow trends loaded from file.')
    
snowlines_medians
        

In [ ]:
# -----Compile RGI characteristics and minimum snow cover median statistics
min_snow_cover_stats_fn = os.path.join(scm_path, 'results', 'min_snow_cover_stats.csv') 
# check if exists in directory
if not os.path.exists(min_snow_cover_stats_fn):
    # initialize dataframe for RGI stats and minimum snow cover statts
    min_snow_cover_stats = pd.DataFrame()
    
    # iterate over site names in median snowlines dataframe
    for site_name in tqdm(sorted(snowlines_medians['site_name'].drop_duplicates().values)):
        # grab AOI for site
        aoi_site = aois.loc[aois['RGIId']==site_name, :]
        # grab median snowline stats for site
        snowlines_medians_site = snowlines_medians.loc[snowlines_medians['site_name']==site_name, :]
        # calculate min median stats
        median_columns = [x for x in snowlines_medians.columns if 'P50' in x]
        for column in median_columns:
            if (column=='ELA_from_AAR_m_P50') or (column=='snowline_elevs_median_m_P50'):
                aoi_site[column+'_max'] = snowlines_medians_site[column].max()
            else:
                aoi_site[column+'_min'] = snowlines_medians_site[column].min()
        # concatenate to full dataframe
        min_snow_cover_stats = pd.concat([min_snow_cover_stats, aoi_site])

    # add subregion names and colors
    min_snow_cover_stats[['Subregion', 'color']] = '', ''
    min_snow_cover_stats[['O1Region', 'O2Region']] = min_snow_cover_stats[['O1Region', 'O2Region']].astype(int)
    for o1, o2 in min_snow_cover_stats[['O1Region', 'O2Region']].drop_duplicates().values:
        min_snow_cover_stats.loc[(min_snow_cover_stats['O1Region']==o1) 
                                 & (min_snow_cover_stats['O2Region']==o2), ['Subregion', 'color']] = f.determine_subregion_name_color(o1, o2)
    # save to file
    min_snow_cover_stats.to_csv(min_snow_cover_stats_fn, index=False)
    print('Minimum median snow cover stats saved to file: ', min_snow_cover_stats_fn)
        
else:
    # load from file
    min_snow_cover_stats = pd.read_csv(min_snow_cover_stats_fn)
    print('Minimum median snow cover stats loaded from file.')

# reformat as GeoDataFrame
min_snow_cover_stats['geometry'] = min_snow_cover_stats['geometry'].apply(wkt.loads)
min_snow_cover_stats = gpd.GeoDataFrame(min_snow_cover_stats, crs='EPSG:4326')
min_snow_cover_stats